# Verge and foliot mechanism

References:

- https://github.com/JuliaReach/ReachabilityModels.jl/issues/52

- https://github.com/SciML/DifferentialEquations.jl/issues/703

## Dependencies

In [1]:
using ReachabilityAnalysis, LinearAlgebra, Plots, IntervalConstraintProgramming, StaticArrays
const IA = IntervalArithmetic

LazySets.set_ztol(Float64, 1e-12)

plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/mforets/.julia/packages/Plots/vsE7b/src/backends.jl:372


Plots.PlotlyBackend()

## Model parameters

In [38]:
const torque = 1.0
const Ic = 10.0
const Iv = 0.15
const rc = 1.0
const rv = 0.3
const αc = 24.0 * pi/180
const er = 0.05
const nteeth = round(Int, 2pi/αc) # n
const Mv = Iv/rv^2
const Mc = Ic/rc^2
const Gc = Mv*(1 + er)/rc/(Mv + Mc)
const Gv = Mc*(1 + er)/rv/(Mv + Mc)
const αv = Gv*rc/(1-er)*αc/2

p = (torque, Ic);

## Guards

In [39]:
# trigonometric constraints for each m
upper_trig(m) = @constraint $rc*sin(x1 - ($m-1)* ($αc)) - $rv*tan(x2 + $αv/2) == 0
lower_trig(m) = @constraint $rc*sin(($m-1)* ($αc) - x1) - $rv*tan(-x2 + $αv/2) == 0

# domain of x1 for each m
upper_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p)*pi
lower_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p-1)*pi

# combined domain of x1 and x2 for each m 
upper_dom(m; p=0) = upper_dom_x1(m, p=p) × (-1 .. 1)
lower_dom(m; p=0) = lower_dom_x1(m, p=p) × (-1 .. 1)

# constraint solver tolerance
ε = 0.05

# upper constraints
U = [pave(upper_trig(m), upper_dom(m, p=0), ε).boundary for m in 1:nteeth]
Uv = [reduce(vcat, vertices_list(B) for B in Ui) |> VPolygon for Ui in U]

# lower constraints
L = [pave(lower_trig(m), lower_dom(m, p=0), ε).boundary for m in 1:nteeth];
Lv = [reduce(vcat, vertices_list(B) for B in Li) |> VPolygon for Li in L];

In [40]:
fig = plot(xlab="x₁", ylab="x₂", title="Collision conditions, blue: upper, red: lower")

plot!(fig, Uv, c=:blue)
plot!(fig, Lv, c=:red)

<!DOCTYPE html>
 
 
 Plots.jl

In [41]:
G34_upper = HalfSpace(SA[-rc, rv], 0.0) # rc x3 - rv x4 > 0
G34_lower = HalfSpace(SA[-rc, -rv], 0.0); # rc x3 + rv x4 > 0

## Transition map

In [42]:
# --------------------
# TODO use SMatrix
# --------------------

# σ = 1
Tupper = SA[[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, -rc*Gc, rv*Gc],
            [0, 0, rc*Gv, -rv*Gv]]

# σ = -1
Tlower = SA[[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [0, 0, -rc*Gc, -rv*Gc],
            [0, 0, -rc*Gv, -rv*Gv]];

## Continuous post-operator

In [43]:
# variables x1, x2, x3, x4 and the last variable x5 wraps the constant input term
A = zeros(5, 5)
A[1, 3] = 1.0
A[2, 4] = 1.0
A[3, 5] = 1.0

X0 = Singleton(SA[0, 0, 0, 3.0, p[1]/p[2]])
prob = @ivp(x' = A*x, x(0) ∈ X0);

In [44]:
using BenchmarkTools

In [45]:
@btime solve($prob, tspan=(0.0, 0.5), alg=GLGM06(δ=1e-3));

  364.863 μs (1173 allocations: 238.31 KiB)


In [46]:
#@btime solve($prob, tspan=(0.0, 0.5), alg=GLGM06(δ=1e-3, static=true, dim=5, ngens=10, max_order=2))
#  234.797 μs (229 allocations: 277.86 KiB)

In [47]:
plot(sol, vars=(0, 1), xlab="t", ylab="x₁(t)")

<!DOCTYPE html>
 
 
 Plots.jl

In [48]:
plot(sol, vars=(3, 4), xlab="x₃", ylab="x₄")

LoadError: ArgumentError: the variables vars do not belong to the variables  of this reach-set, (1, 2)

In [49]:
fig = plot(xlab="x₁", ylab="x₂", title="Trig constraints (p = 0), blue: upper, red: lower")

plot!(fig, sol, vars=(1, 2), xlab="x₁", ylab="x₂", lw=1.0, alpha=1.0)
plot!(fig, Uv, c=:blue)
plot!(fig, Lv, c=:red)

<!DOCTYPE html>
 
 
 Plots.jl

In [50]:
fig = plot(xlab="x₃", ylab="x₄", xlims=(-10, 10), ylims=(-10, 10))

plot!(fig, G34_upper, lab="Upper", alpha=.4)
plot!(fig, G34_lower, lab="Lower",  alpha=.4)

# see ReachabilityAnalysis#389
plot!(fig, [project(set(R), (3, 4)) for R in sol], lw=3.0)

LoadError: MethodError: no method matching project(::Array{TaylorModels.TaylorModel1{TaylorN{Float64},Float64},1}, ::Tuple{Int64,Int64})
Closest candidates are:
  project(!Matched::SubArray{var"#s24",N,P,I,L} where L where I where P where N where var"#s24"<:ReachabilityAnalysis.AbstractLazyReachSet, ::Any) at /home/mforets/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:306
  project(!Matched::ReachabilityAnalysis.AbstractLazyReachSet, ::Tuple{Vararg{M,D}}; check_vars) where {D, M<:Integer} at /home/mforets/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:489
  project(!Matched::TaylorModelReachSet, ::Tuple{Vararg{M,D}}) where {D, M<:Integer} at /home/mforets/.julia/dev/ReachabilityAnalysis/src/Flowpipes/reachsets.jl:637
  ...

## Algorithm

In [51]:
fig = plot(xlab="x₁", ylab="x₂", title="Trig constraints (p = 0), blue: upper, red: lower")

plot!(fig, sol, vars=(1, 2), xlab="x₁", ylab="x₂", lw=1.0, alpha=1.0)
plot!(fig, Uv, c=:blue)
plot!(fig, Lv, c=:red)

<!DOCTYPE html>
 
 
 Plots.jl

In [52]:
# find guard's index which intersect the flowpipe
#sol12 = project(sol, (1, 2))

typeof(sol)

ReachabilityAnalysis.ReachSolution{Flowpipe{Float64,TaylorModelReachSet{Float64},Array{TaylorModelReachSet{Float64},1}},TMJets{Float64,ZonotopeEnclosure}}

In [53]:
using Revise, ReachabilityAnalysis, Test, BenchmarkTools

In [54]:
function harmonic_oscillator(; X=Universe(2))
    A = [0.0 1.0; -1.0 0.0]
    prob = @ivp(x' = Ax, x(0) ∈ Singleton([1.0, 0.0]), x ∈ X)
    tspan = (0.0, 20.0)
    return prob, tspan
end


prob, dt = harmonic_oscillator()
sol = solve(prob, tspan=dt)
fp = flowpipe(sol);

In [55]:
x = rand(Hyperrectangle, dim=1)

Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([-1.4745608099356677], [0.27293745282230075])

In [ ]:
project(fp, (0, 1))

In [ ]:
project(sol, (1, 2))

In [ ]:
Projection(sol, (1, 2))

In [ ]:
typeof(Lv[1])

In [ ]:
typeof(sol12)

In [ ]:
isdisjoint(sol, Lv[1])

In [ ]:
# find guard's index which intersect the flowpipe
sol12 = project(sol, (1, 2))

findall(Gm -> !isdisjoint(sol12, Gm), Lv)

In [ ]:
# using loops
sol12 = project(sol, (1, 2)) # currently returns an array of LazySets
sol12u = UnionSetArray(sol12);

In [ ]:
# find all
@time findall(X -> !isdisjoint(sol12u, X), Lv);

In [ ]:
plot(Lv[9])
plot!(sol, vars=(1, 2))

In [ ]:
findall(X -> !isdisjoint(X, Lv[9]) && !(X ⊆ Lv[9]), array(sol12u))

In [ ]:
plot(Lv[9])
plot!(sol, vars=(1, 2))
plot!(sol[295], vars=(1, 2), c=:black)
plot!(sol[313], vars=(1, 2), c=:black)

In [ ]:
plot(Lv[9])
plot!(sol, vars=(1, 2))
plot!(sol[295], vars=(1, 2), c=:black)
plot!(sol[313], vars=(1, 2), c=:black)
Y = intersection(sol12[295], Lv[9])
plot!(Y)

In [ ]:
plot(Lv[9])
plot!(sol, vars=(1, 2))
plot!(sol[295], vars=(1, 2), c=:black)
plot!(sol[313], vars=(1, 2), c=:black)
Y = intersection(sol12[295], Lv[9])
plot!(Y)

Z = overapproximate(Y, Zonotope, OctDirections)
plot!(Z)

In [ ]:
Z = overapproximate(Y, Zonotope, PolarDirections(20))

In [ ]:
plot(Y, lab="Y")
plot!(Z, lab="Z")

In [ ]:
Z.generators

In [ ]:
#overapproximate(sol12[295] ∩ Lv[9], Zonotope) # not working
#overapproximate(sol12[295] ∩ Lv[9], Zonotope, BoxDirections(2)) # not working


In [ ]:
struct Clock{T<:AbstractContinuousSystem, MT, CT, VT} <: AbstractHybridSystem
    fc::T      # continuous system
    fd::MT     # jump function 
    Cupper::CT # collision condition (upper part)
    Clower::CT # collision condition (lower part)
    Vupper::VT # velocities condition (upper part)
    Vlower::VT # lower part
end

In [ ]:
const RA = ReachabilityAnalysis

In [ ]:
function _solve(ivp::IVP{<:VergeAndFoliot};
                tspan, alg,
                max_jumps=1000, # maximum number of discrete transitions
                intersection_method::AbstractIntersectionMethod=HRepIntersection(), # method to take the concrete intersection in discrete transitions
                clustering_method::AbstractClusteringMethod=BoxClustering(), # method to perform clustering of the sets that cross a guard
                check_invariant_initial_states=false, # apply a disjointness check wrt each mode's invariant in the distribution of initial sets
                intersect_invariant_initial_states=false, # take the concrete intersection wrt each mode's invariant when distributing the initial states
                intersection_source_invariant_method=FallbackIntersection(), # method to take the concrete intersection with the source invariant
                intersect_source_invariant=true, # take the concrete intersection of the flowpipe with the source invariant
                disjointness_method=NoEnclosure(), # method to compute disjointness
                fixpoint_check=true) # if true, stop the integration when a fix point is detected

    X0 = initial_state(ivp)
    loc = 1 # only one location
    STwl = Zonotope{Vector{Float64}, Matrix{Float64}}
    waiting_list = WaitingList{TimeInterval, STwl, Int}()
    push!(waiting_list, StateInLocation(X0, loc))

    H = system(ivp)

    time_span0 = time_span

    # get the continuous post or use a default one

    # preallocate output flowpipe
    N = numtype(cpost)
    RT = rsetrep(cpost)
    out = Vector{Flowpipe{N, RT, Vector{RT}}}()
    sizehint!(out, max_jumps+1)

    # list of (set, loc) tuples which have already been processed
    explored_list = typeof(waiting_list)()

    # elapsed time accumulators
    t0 = tstart(time_span)
    T = tend(time_span) # time horizon

    # counter for the number of transitions: using `count_jumps <= max_jumps` as
    # stopping criterion ensures that no more elements are added to the waiting
    # list after `max_jumps` discrete jumps
    count_jumps = 0

    while !isempty(waiting_list)
        (Δt0, elem) = pop!(waiting_list)
        tprev = tstart(Δt0)
        push!(explored_list, elem)

        # compute reachable states by continuous evolution
        X0 = state(elem)
        time_span = TimeInterval(t0, T-tprev)
        F = post(cpost, IVP(S, X0), time_span; Δt0=Δt0, kwargs...)

        push!(out, F)

        #=
        # NOTE: here we may take the concrete intersection with the source invariant
        # We have to loop over each reach-set in F and only store the intersection
        # (possibly overapproximated) with the source invariant I⁻
        Two ways: (i) compute concrete intersection between F and I⁻
        (ii) compute disjointness between F and (I⁻)^C and only when it is non-zero
        compute the concrete intersection.
        =#
        if intersect_source_invariant
            I⁻ = stateset(H, q)

            # assign location q to this flowpipe
            F_in_inv = Flowpipe(undef, STwl, 0)

            for Ri in F
                # TODO refactor with reconstruct
                aux = _intersection(Ri, I⁻, intersection_source_invariant_method)
                Raux = ReachSet(aux, tspan(Ri))
                push!(F_in_inv, Raux)
            end
            F_in_inv.ext[:loc_id] = q
            push!(out_in_inv, F_in_inv)
        end

        # process jumps for all outgoing transitions with source q
        for t in out_transitions(H, q)

            # instantiate post_d : X -> (R(X ∩ G ∩ I⁻) ⊕ W) ∩ I⁺
            discrete_post = DiscreteTransition(H, t)
            G = guard(discrete_post)

            # find reach-sets that may take the jump
            jump_rset_idx = Vector{Int}()
            for (i, X) in enumerate(F)
                _is_intersection_empty(X, G, disjointness_method) && continue
                push!(jump_rset_idx, i)
            end

            # continue if there is no guard enabled for this transition
            isempty(jump_rset_idx) && continue

            # apply clustering method to those sets which intersect the guard
            Xc = cluster(F, jump_rset_idx, clustering_method)

            for Xci in Xc
                # compute reachable states by discrete evolution
                X = apply(discrete_post, Xci, intersection_method)

                # do not add empty sets; checking `isempty` generalizes
                isa(X, EmptySet) && continue
                #isempty(X) && continue # though it may have to solve a feasibility LP if X is a polyhedron

                count_jumps += 1

                # check if this location has already been explored;
                # if it is not the case, add it to the waiting list
                r = target(H, t)
                Xr = StateInLocation(X, r)

                hit_max_jumps = count_jumps > max_jumps
                if hit_max_jumps
                    @warn "maximum number of jumps reached; try increasing `max_jumps`"
                end

                if fixpoint_check
                    if !hit_max_jumps && !(Xr ⊆ explored_list)
                        push!(waiting_list, tspan(Xci), Xr)
                    end
                else
                    # We only push the set Xci if it intersects with time_span
                    if !hit_max_jumps && !IA.isdisjoint(tspan(Xci), time_span0)
                        push!(waiting_list, tspan(Xci), Xr)
                    end
                end
            end
        end # for
    end # while

    # wrap the flowpipe and algorithm in a solution structure
    if intersect_source_invariant
        HFout = HybridFlowpipe(out_in_inv)
    else
        HFout = HybridFlowpipe(out)
    end
    sol = ReachSolution(HFout, cpost)
end